In [13]:
import os
import pandas as pd
import numpy as np
import lightgbm as lgbm
from sklearn import preprocessing
from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score
from hyperopt import hp
from sklearn.model_selection import KFold
from hyperopt import fmin, tpe, Trials
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

# 현재 작업 디렉토리에서 모든 CSV 파일의 이름을 가져옴
csv_files = [f for f in os.listdir() if f.endswith('.csv')]

# 모든 CSV 파일을 하나의 데이터프레임으로 병합
baseball_data = pd.concat([pd.read_csv(f) for f in csv_files])

In [14]:
# 필요없는 항목 삭제
baseball_data = baseball_data.drop(columns=['G_ID','GDAY_DS','HEADER_NO','TB_SC','P_HRA_RT','P_AB_CN','P_HIT_CN','OOO','OBP'])
# 무승부 제거
baseball_data = baseball_data[baseball_data['win']!=0.5]

In [15]:
# LabelEncoder로 홈팀, 상대팀 정수 변환
le = preprocessing.LabelEncoder()
baseball_data['T_ID'] = le.fit_transform(baseball_data['T_ID'])
baseball_data['VS_T_ID'] = le.fit_transform(baseball_data['VS_T_ID'])

In [16]:
# 정수 처리
baseball_data = baseball_data.astype(int)

In [17]:
X_features = baseball_data.iloc[:, :-1]
y_labels = baseball_data.iloc[:, -1]
print('피처 데이터 shape:{0}'.format(X_features.shape))

피처 데이터 shape:(8522, 18)


In [18]:
X_train, X_test, y_train, y_test = train_test_split(X_features, y_labels,
                                                    test_size=0.2, random_state=0)
train_cnt = y_train.count()
test_cnt = y_test.count()
print('학습 세트 Shape:{0}, 테스트 세트 Shape:{1}'.format(X_train.shape , X_test.shape))

print(' 학습 세트 레이블 값 분포 비율')
print(y_train.value_counts()/train_cnt)
print('\n 테스트 세트 레이블 값 분포 비율')
print(y_test.value_counts()/test_cnt)

학습 세트 Shape:(6817, 18), 테스트 세트 Shape:(1705, 18)
 학습 세트 레이블 값 분포 비율
0    0.501687
1    0.498313
Name: win, dtype: float64

 테스트 세트 레이블 값 분포 비율
1    0.506745
0    0.493255
Name: win, dtype: float64


In [19]:
# X_train, y_train을 다시 학습과 검증 데이터 세트로 분리. 
X_tr, X_val, y_tr, y_val = train_test_split(X_train, y_train,
                                                    test_size=0.3, random_state=0)

In [21]:
lgbm_clf = LGBMClassifier(n_estimators=500)

eval_set=[(X_tr, y_tr), (X_val, y_val)]
lgbm_clf.fit(X_tr, y_tr, early_stopping_rounds=100, eval_metric="auc", eval_set=eval_set)

lgbm_roc_score = roc_auc_score(y_test, lgbm_clf.predict_proba(X_test)[:,1])
print('ROC AUC: {0:.4f}'.format(lgbm_roc_score))

/usr/local/lib/python3.9/dist-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "


[1]	training's auc: 0.886765	training's binary_logloss: 0.648908	valid_1's auc: 0.871856	valid_1's binary_logloss: 0.650888
[2]	training's auc: 0.891717	training's binary_logloss: 0.61227	valid_1's auc: 0.874441	valid_1's binary_logloss: 0.616893
[3]	training's auc: 0.894889	training's binary_logloss: 0.581871	valid_1's auc: 0.875095	valid_1's binary_logloss: 0.589296
[4]	training's auc: 0.900856	training's binary_logloss: 0.55547	valid_1's auc: 0.879746	valid_1's binary_logloss: 0.565305
[5]	training's auc: 0.902766	training's binary_logloss: 0.533593	valid_1's auc: 0.881804	valid_1's binary_logloss: 0.545544
[6]	training's auc: 0.90577	training's binary_logloss: 0.513365	valid_1's auc: 0.884221	valid_1's binary_logloss: 0.527566
[7]	training's auc: 0.907438	training's binary_logloss: 0.496115	valid_1's auc: 0.886015	valid_1's binary_logloss: 0.511854
[8]	training's auc: 0.908941	training's binary_logloss: 0.481052	valid_1's auc: 0.885908	valid_1's binary_logloss: 0.49981
[9]	training

In [22]:
lgbm_search_space = {'num_leaves': hp.quniform('num_leaves', 32, 64, 1),
                     'max_depth': hp.quniform('max_depth', 100, 160, 1),
                     'min_child_samples': hp.quniform('min_child_samples', 60, 100, 1),
                     'subsample': hp.uniform('subsample', 0.7, 1),
                     'learning_rate': hp.uniform('learning_rate', 0.01, 0.2)

                    }

In [23]:
def objective_func(search_space):
    lgbm_clf =  LGBMClassifier(n_estimators=100, num_leaves=int(search_space['num_leaves']),
                               max_depth=int(search_space['max_depth']),
                               min_child_samples=int(search_space['min_child_samples']), 
                               subsample=search_space['subsample'],
                               learning_rate=search_space['learning_rate'])
    # 3개 k-fold 방식으로 평가된 roc_auc 지표를 담는 list
    roc_auc_list = []
    
    # 3개 k-fold방식 적용 
    kf = KFold(n_splits=3)
    # X_train을 다시 학습과 검증용 데이터로 분리
    for tr_index, val_index in kf.split(X_train):
        # kf.split(X_train)으로 추출된 학습과 검증 index값으로 학습과 검증 데이터 세트 분리 
        X_tr, y_tr = X_train.iloc[tr_index], y_train.iloc[tr_index]
        X_val, y_val = X_train.iloc[val_index], y_train.iloc[val_index]

        # early stopping은 30회로 설정하고 추출된 학습과 검증 데이터로 XGBClassifier 학습 수행. 
        lgbm_clf.fit(X_tr, y_tr, early_stopping_rounds=30, eval_metric="auc",
           eval_set=[(X_tr, y_tr), (X_val, y_val)])

        # 1로 예측한 확률값 추출후 roc auc 계산하고 평균 roc auc 계산을 위해 list에 결과값 담음.
        score = roc_auc_score(y_val, lgbm_clf.predict_proba(X_val)[:, 1]) 
        roc_auc_list.append(score)
    
    # 3개 k-fold로 계산된 roc_auc값의 평균값을 반환하되, 
    # HyperOpt는 목적함수의 최소값을 위한 입력값을 찾으므로 -1을 곱한 뒤 반환.
    return -1*np.mean(roc_auc_list)

In [24]:
trials = Trials()

# fmin()함수를 호출. max_evals지정된 횟수만큼 반복 후 목적함수의 최소값을 가지는 최적 입력값 추출. 
best = fmin(fn=objective_func, space=lgbm_search_space, algo=tpe.suggest,
            max_evals=50, # 최대 반복 횟수를 지정합니다.
            trials=trials, rstate=np.random.default_rng(seed=30))

print('best:', best)

[1]	training's auc: 0.885488	training's binary_logloss: 0.682645	valid_1's auc: 0.876926	valid_1's binary_logloss: 0.683029
[2]	training's auc: 0.885808	training's binary_logloss: 0.672624	valid_1's auc: 0.878085	valid_1's binary_logloss: 0.673251
[3]	training's auc: 0.886753	training's binary_logloss: 0.663112	valid_1's auc: 0.87895	valid_1's binary_logloss: 0.664056
[4]	training's auc: 0.887816	training's binary_logloss: 0.654148	valid_1's auc: 0.879648	valid_1's binary_logloss: 0.655347
[5]	training's auc: 0.887403	training's binary_logloss: 0.645589	valid_1's auc: 0.879971	valid_1's binary_logloss: 0.647026
[6]	training's auc: 0.887735	training's binary_logloss: 0.637248	valid_1's auc: 0.879898	valid_1's binary_logloss: 0.638994
[7]	training's auc: 0.891689	training's binary_logloss: 0.629112	valid_1's auc: 0.882598	valid_1's binary_logloss: 0.631269
[8]	training's auc: 0.892678	training's binary_logloss: 0.621408	valid_1's auc: 0.883839	valid_1's binary_logloss: 0.623874
[9]	train

/usr/local/lib/python3.9/dist-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "



[20]	training's auc: 0.898808	training's binary_logloss: 0.548219	valid_1's auc: 0.888013	valid_1's binary_logloss: 0.554164
[21]	training's auc: 0.899293	training's binary_logloss: 0.543326	valid_1's auc: 0.888153	valid_1's binary_logloss: 0.549567
[22]	training's auc: 0.900008	training's binary_logloss: 0.538449	valid_1's auc: 0.888385	valid_1's binary_logloss: 0.545178
[23]	training's auc: 0.900412	training's binary_logloss: 0.533863	valid_1's auc: 0.888966	valid_1's binary_logloss: 0.540886
[24]	training's auc: 0.901008	training's binary_logloss: 0.529374	valid_1's auc: 0.889033	valid_1's binary_logloss: 0.536823
[25]	training's auc: 0.901259	training's binary_logloss: 0.525096	valid_1's auc: 0.889293	valid_1's binary_logloss: 0.5328
[26]	training's auc: 0.901663	training's binary_logloss: 0.520844	valid_1's auc: 0.889224	valid_1's binary_logloss: 0.529081
[27]	training's auc: 0.901951	training's binary_logloss: 0.516843	valid_1's auc: 0.88961	valid_1's binary_logloss: 0.525319
[28

/usr/local/lib/python3.9/dist-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "



[17]	training's auc: 0.893031	training's binary_logloss: 0.5655	valid_1's auc: 0.885302	valid_1's binary_logloss: 0.567083
[18]	training's auc: 0.89451	training's binary_logloss: 0.559991	valid_1's auc: 0.887071	valid_1's binary_logloss: 0.561738
[19]	training's auc: 0.895185	training's binary_logloss: 0.554867	valid_1's auc: 0.888245	valid_1's binary_logloss: 0.556684
[20]	training's auc: 0.895911	training's binary_logloss: 0.549845	valid_1's auc: 0.888934	valid_1's binary_logloss: 0.551826
[21]	training's auc: 0.896086	training's binary_logloss: 0.545155	valid_1's auc: 0.889124	valid_1's binary_logloss: 0.547188
[22]	training's auc: 0.896949	training's binary_logloss: 0.540318	valid_1's auc: 0.889961	valid_1's binary_logloss: 0.542485
[23]	training's auc: 0.897402	training's binary_logloss: 0.53571	valid_1's auc: 0.890524	valid_1's binary_logloss: 0.537987
[24]	training's auc: 0.897467	training's binary_logloss: 0.531458	valid_1's auc: 0.890736	valid_1's binary_logloss: 0.5338
[25]	t

...

[12]	training's auc: 0.91163	training's binary_logloss: 0.408159	valid_1's auc: 0.895025	valid_1's binary_logloss: 0.429586
[13]	training's auc: 0.914115	training's binary_logloss: 0.400141	valid_1's auc: 0.896074	valid_1's binary_logloss: 0.424276
[14]	training's auc: 0.916006	training's binary_logloss: 0.393521	valid_1's auc: 0.897026	valid_1's binary_logloss: 0.419768
[15]	training's auc: 0.918372	training's binary_logloss: 0.386615	valid_1's auc: 0.898662	valid_1's binary_logloss: 0.414451
[16]	training's auc: 0.920133	training's binary_logloss: 0.380825	valid_1's auc: 0.899207	valid_1's binary_logloss: 0.411336
[17]	training's auc: 0.922363	training's binary_logloss: 0.374968	valid_1's auc: 0.900335	valid_1's binary_logloss: 0.40781
[18]	training's auc: 0.924698	training's binary_logloss: 0.369102	valid_1's auc: 0.901426	valid_1's binary_logloss: 0.404767
[19]	training's auc: 0.926359	training's binary_logloss: 0.364238	valid_1's auc: 0.901497	valid_1's binary_logloss: 0.403129
[2

/usr/local/lib/python3.9/dist-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "



[5]	training's auc: 0.894909	training's binary_logloss: 0.498155	valid_1's auc: 0.890329	valid_1's binary_logloss: 0.499755
[6]	training's auc: 0.897287	training's binary_logloss: 0.479621	valid_1's auc: 0.89084	valid_1's binary_logloss: 0.482712
[7]	training's auc: 0.899335	training's binary_logloss: 0.46417	valid_1's auc: 0.892108	valid_1's binary_logloss: 0.469079
[8]	training's auc: 0.90181	training's binary_logloss: 0.450247	valid_1's auc: 0.89331	valid_1's binary_logloss: 0.456549
[9]	training's auc: 0.903821	training's binary_logloss: 0.4386	valid_1's auc: 0.893655	valid_1's binary_logloss: 0.447752
[10]	training's auc: 0.906355	training's binary_logloss: 0.427764	valid_1's auc: 0.894693	valid_1's binary_logloss: 0.439422
[11]	training's auc: 0.908573	training's binary_logloss: 0.418605	valid_1's auc: 0.895412	valid_1's binary_logloss: 0.432919
[12]	training's auc: 0.91079	training's binary_logloss: 0.409875	valid_1's auc: 0.89649	valid_1's binary_logloss: 0.427046
[13]	training

/usr/local/lib/python3.9/dist-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "



[10]	training's auc: 0.912129	training's binary_logloss: 0.41872	valid_1's auc: 0.883454	valid_1's binary_logloss: 0.453163
[11]	training's auc: 0.913608	training's binary_logloss: 0.409859	valid_1's auc: 0.884379	valid_1's binary_logloss: 0.446782
[12]	training's auc: 0.914861	training's binary_logloss: 0.402349	valid_1's auc: 0.884662	valid_1's binary_logloss: 0.442188
[13]	training's auc: 0.917042	training's binary_logloss: 0.394268	valid_1's auc: 0.886167	valid_1's binary_logloss: 0.436781
[14]	training's auc: 0.918264	training's binary_logloss: 0.388125	valid_1's auc: 0.886838	valid_1's binary_logloss: 0.4332
[15]	training's auc: 0.920153	training's binary_logloss: 0.381791	valid_1's auc: 0.887924	valid_1's binary_logloss: 0.429422
[16]	training's auc: 0.922745	training's binary_logloss: 0.374865	valid_1's auc: 0.889338	valid_1's binary_logloss: 0.425558
[17]	training's auc: 0.924635	training's binary_logloss: 0.369136	valid_1's auc: 0.89077	valid_1's binary_logloss: 0.421881
[18]

In [26]:
lgbm_clf =  LGBMClassifier(n_estimators=500, num_leaves=int(best['num_leaves']),
                           max_depth=int(best['max_depth']),
                           min_child_samples=int(best['min_child_samples']), 
                           subsample=round(best['subsample'], 5),
                           learning_rate=round(best['learning_rate'], 5)
                          )

# evaluation metric을 auc로, early stopping은 100 으로 설정하고 학습 수행. 
lgbm_clf.fit(X_tr, y_tr, early_stopping_rounds=100, 
            eval_metric="auc",eval_set=[(X_tr, y_tr), (X_val, y_val)])

lgbm_roc_score = roc_auc_score(y_test, lgbm_clf.predict_proba(X_test)[:,1])
print('ROC AUC: {0:.4f}'.format(lgbm_roc_score))

/usr/local/lib/python3.9/dist-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "


[1]	training's auc: 0.882007	training's binary_logloss: 0.642154	valid_1's auc: 0.866491	valid_1's binary_logloss: 0.644248
[2]	training's auc: 0.887154	training's binary_logloss: 0.601461	valid_1's auc: 0.867789	valid_1's binary_logloss: 0.606418
[3]	training's auc: 0.890425	training's binary_logloss: 0.568496	valid_1's auc: 0.869494	valid_1's binary_logloss: 0.575613
[4]	training's auc: 0.892172	training's binary_logloss: 0.542202	valid_1's auc: 0.873501	valid_1's binary_logloss: 0.550963
[5]	training's auc: 0.895018	training's binary_logloss: 0.519705	valid_1's auc: 0.878176	valid_1's binary_logloss: 0.528842
[6]	training's auc: 0.898124	training's binary_logloss: 0.500416	valid_1's auc: 0.879882	valid_1's binary_logloss: 0.511803
[7]	training's auc: 0.899973	training's binary_logloss: 0.48373	valid_1's auc: 0.881012	valid_1's binary_logloss: 0.497569
[8]	training's auc: 0.902423	training's binary_logloss: 0.468987	valid_1's auc: 0.883313	valid_1's binary_logloss: 0.484261
[9]	train

In [27]:
from sklearn.metrics import accuracy_score

y_pred = lgbm_clf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.8029325513196481


In [28]:
print('ROC AUC: {0:.4f}'.format(lgbm_roc_score))

ROC AUC: 0.8958


In [29]:
print('Accuracy: {0:.4f}'.format(accuracy))

Accuracy: 0.8029


In [42]:
# 0일 확률
pred_probs = lgbm_clf.predict_proba(X_test)[:, 0]
print('Probability of class label 0 for the first sample:', pred_probs[-1])

Probability of class label 0 for the first sample: 0.9863712288222929


In [38]:
# 0일 확률
pred_probs = lgbm_clf.predict_proba(X_test)[:, 1]
print('Probability of class label 0 for the first sample:', pred_probs[0])

Probability of class label 0 for the first sample: 0.011439955710511552
